In [1]:
import numpy as np
import rasterio
from rasterio.transform import from_origin
from rasterio.crs import CRS

# Parameters for random raster
width, height = 100, 100
count = 1
dtype = 'float32'
crs = CRS.from_epsg(25833)
transform = from_origin(500000, 7000000, 10, 10)  # arbitrary origin and 10x10m pixels

for i in range(3):
    data = np.random.rand(height, width).astype(dtype)
    filename = f"random_geotiff_{i+1}.tif"
    with rasterio.open(
        filename,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=count,
        dtype=dtype,
        crs=crs,
        transform=transform
    ) as dst:
        dst.write(data, 1)
    print(f"Saved {filename}")

Saved random_geotiff_1.tif
Saved random_geotiff_2.tif
Saved random_geotiff_3.tif


In [5]:
import os
import numpy as np
import rasterio
from rasterio.enums import Resampling
from rasterio.shutil import copy as rio_copy
from rasterio.features import shapes
from rasterio.mask import mask as rasterio_mask
import geopandas as gpd
from shapely.geometry import shape, Polygon, box
from concurrent.futures import ThreadPoolExecutor
import warnings
import pandas as pd

def find_tiffs(folder):
    """Recursively find all .tif or .tiff files in a folder."""
    tiffs = []
    for root, _, files in os.walk(folder):
        for f in files:
            if f.lower().endswith(('.tif', '.tiff')):
                tiffs.append(os.path.join(root, f))
    return tiffs

def convert_to_cog(src_path, dst_path):
    """Convert a raster to COG with LZW compression and pyramids."""
    rio_copy(
        src_path,
        dst_path,
        driver='COG',
        compress='LZW',
        overview_resampling=Resampling.nearest,
        overview_levels=[2, 4, 8, 16]
    )

def get_raster_metadata(raster_path):
    """Get spatial metadata from a raster file."""
    with rasterio.open(raster_path) as src:
        return {
            'crs': src.crs,
            'transform': src.transform,
            'width': src.width,
            'height': src.height,
            'bounds': src.bounds
        }

def check_spatial_alignment(raster_path, reference_meta):
    """Check if raster aligns with reference metadata."""
    current_meta = get_raster_metadata(raster_path)
    
    alignment_check = (
        current_meta['crs'] == reference_meta['crs'] and
        current_meta['transform'] == reference_meta['transform'] and
        current_meta['width'] == reference_meta['width'] and
        current_meta['height'] == reference_meta['height']
    )
    
    return alignment_check, current_meta

def geometries_almost_equal(geom1, geom2, tolerance=1e-6):
    """Check if two geometries are approximately equal within a tolerance."""
    try:
        # Check if geometries are exactly equal first
        if geom1.equals(geom2):
            return True
        
        # Check if they have the same type
        if type(geom1) != type(geom2):
            return False
        
        # Check if centroids are close
        centroid_distance = geom1.centroid.distance(geom2.centroid)
        if centroid_distance > tolerance:
            return False
        
        # Check if areas are similar (within 1% tolerance)
        area_diff = abs(geom1.area - geom2.area) / max(geom1.area, geom2.area, 1e-10)
        if area_diff > 0.01:
            return False
        
        # Check if they have high intersection overlap
        if geom1.intersects(geom2):
            intersection_area = geom1.intersection(geom2).area
            overlap_ratio = intersection_area / min(geom1.area, geom2.area)
            return overlap_ratio > 0.95
        
        return False
    except Exception:
        return False

def create_pixel_polygons(raster_data, transform, mask_array):
    """
    Create individual polygon for each valid pixel to ensure consistent spatial structure.
    
    This prevents merging of adjacent pixels with same values, which is crucial for
    incremental processing where subsequent rasters may have different value distributions.
    
    Optimized version using vectorized operations and shapely.box for better performance.
    """
    height, width = raster_data.shape
    pixel_width = abs(transform.a)
    pixel_height = abs(transform.e)
    
    # Find valid (unmasked) pixel coordinates
    valid_rows, valid_cols = np.where(mask_array)
    
    if len(valid_rows) == 0:
        return [], [], []
    
    # Vectorized coordinate calculations
    x_mins = transform.c + valid_cols * transform.a
    y_maxs = transform.f + valid_rows * transform.e
    
    # Optimized polygon creation using shapely.box (faster for rectangles)
    geometries = [
        box(x_min, y_max - pixel_height, x_min + pixel_width, y_max)
        for x_min, y_max in zip(x_mins, y_maxs)
    ]
    
    # Vectorized value and index extraction
    values = raster_data[valid_rows, valid_cols].astype(float).tolist()
    pixel_indices = (valid_rows * width + valid_cols).tolist()
    
    return geometries, values, pixel_indices

def raster_to_vector_ordered(raster_path, mask_value=None, simplify_tolerance=None, source_filename=None):
    """
    Convert raster to vector polygons maintaining consistent polygon order.
    
    Creates individual polygons for each pixel to ensure consistent spatial structure
    across aligned rasters. This prevents merging of adjacent pixels with same values.
    
    Returns polygons in the same order as raster values are read (row-major order).
    This ensures consistent ordering for aligned rasters.
    """
    with rasterio.open(raster_path) as src:
        raster_data = src.read(1)
        transform = src.transform
        crs = src.crs
        nodata = src.nodata
    
    # Create mask efficiently
    if mask_value is not None:
        mask_array = raster_data != mask_value
    elif nodata is not None:
        mask_array = raster_data != nodata
    else:
        mask_array = np.ones_like(raster_data, dtype=bool)
    
    # Check for NoData and issue warning for incremental processing
    masked_pixels = np.sum(~mask_array)
    total_pixels = raster_data.size
    if masked_pixels > 0:
        print(f"  Warning: {masked_pixels}/{total_pixels} pixels masked as NoData in {source_filename}")
    
    # Create individual pixel polygons to prevent merging
    geometries, values, pixel_indices = create_pixel_polygons(raster_data, transform, mask_array)
    
    if not geometries:
        return gpd.GeoDataFrame(columns=['geometry'], crs=crs)
    
    # Create GeoDataFrame - already in row-major order
    gdf_data = {
        'geometry': geometries,
        'value': values,
        'pixel_index': pixel_indices  # Keep for debugging/verification
    }
    
    if source_filename:
        gdf_data['source_file'] = source_filename
    
    gdf = gpd.GeoDataFrame(gdf_data, crs=crs)
    
    print(f"  Created {len(gdf)} individual pixel polygons (no merging)")
    
    return gdf

def extract_raster_values_ordered(raster_path, base_gdf, mask_value=None):
    """
    Extract values from raster for each polygon in base_gdf, maintaining order.
    
    For spatially aligned rasters, this is much faster than spatial matching.
    Uses pixel indices for precise correspondence.
    """
    with rasterio.open(raster_path) as src:
        raster_data = src.read(1)
        transform = src.transform
        nodata = src.nodata
    
    values = []
    
    # Use pixel_index if available for precise mapping
    if 'pixel_index' in base_gdf.columns:
        height, width = raster_data.shape
        
        for pixel_idx in base_gdf['pixel_index']:
            row = pixel_idx // width
            col = pixel_idx % width
            
            if 0 <= row < height and 0 <= col < width:
                value = float(raster_data[row, col])
                # Check for mask values
                if mask_value is not None and value == mask_value:
                    values.append(np.nan)
                elif nodata is not None and value == nodata:
                    values.append(np.nan)
                else:
                    values.append(value)
            else:
                values.append(np.nan)
    else:
        # Fallback to centroid method
        for geom in base_gdf.geometry:
            try:
                # Get centroid and sample
                centroid = geom.centroid
                col = int((centroid.x - transform.c) / transform.a)
                row = int((centroid.y - transform.f) / transform.e)
                
                # Ensure coordinates are within bounds
                if 0 <= row < raster_data.shape[0] and 0 <= col < raster_data.shape[1]:
                    value = float(raster_data[row, col])
                    # Check for mask value
                    if mask_value is not None and value == mask_value:
                        values.append(np.nan)
                    elif nodata is not None and value == nodata:
                        values.append(np.nan)
                    else:
                        values.append(value)
                else:
                    values.append(np.nan)
            except Exception:
                values.append(np.nan)
    
    return values

def raster_to_vector(raster_path, output_path=None, mask_value=None, simplify_tolerance=None, source_filename=None):
    """
    Optimized conversion of raster to vector polygons.
    
    Parameters:
    - raster_path: Path to raster file or numpy array
    - output_path: Output path for vector file (optional)
    - mask_value: Value to mask out
    - simplify_tolerance: Tolerance for polygon simplification (meters)
    - source_filename: Name of source file to add as attribute
    
    Returns:
    - GeoDataFrame with polygons
    """
    if isinstance(raster_path, (str, os.PathLike)):
        # Read from file with optimized reading
        with rasterio.open(raster_path) as src:
            # Read all bands at once if multiband
            if src.count > 1:
                raster_data = src.read()  # Shape: (bands, height, width)
                is_multiband = True
                reference_band = raster_data[0]
            else:
                raster_data = src.read(1)
                is_multiband = False
                reference_band = raster_data
            
            transform = src.transform
            crs = src.crs
            
    elif isinstance(raster_path, np.ndarray):
        raster_data = raster_path
        is_multiband = raster_data.ndim == 3
        
        if is_multiband:
            reference_band = raster_data[0]
        else:
            reference_band = raster_data
            
        # Default transform and CRS for array input
        transform = rasterio.transform.from_bounds(0, 0, reference_band.shape[1], reference_band.shape[0], 
                                                 reference_band.shape[1], reference_band.shape[0])
        crs = 'EPSG:4326'
    else:
        raise ValueError("raster_path must be a file path or numpy array")
    
    # Create mask efficiently
    if mask_value is not None:
        mask_array = reference_band != mask_value
    else:
        mask_array = np.ones_like(reference_band, dtype=bool)
    
    # Extract shapes with connectivity for better polygon extraction
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        polygon_generator = shapes(reference_band, mask=mask_array, transform=transform, connectivity=8)
    
    # Process polygons more efficiently
    geometries = []
    polygon_values = []
    
    for geom_dict, value in polygon_generator:
        geom = shape(geom_dict)
        
        # Optional simplification for performance
        if simplify_tolerance and hasattr(geom, 'simplify'):
            geom = geom.simplify(simplify_tolerance, preserve_topology=True)
        
        geometries.append(geom)
        polygon_values.append(value)
    
    if not geometries:
        # Return empty GeoDataFrame if no polygons found
        return gpd.GeoDataFrame(columns=['geometry'], crs=crs)
    
    # Create GeoDataFrame more efficiently
    if is_multiband:
        # Optimized multi-band value extraction using vectorized operations
        gdf_data = {'geometry': geometries}
        
        # Use rasterio.mask for efficient value extraction per polygon
        for band_idx in range(raster_data.shape[0]):
            band_values = []
            current_band = raster_data[band_idx] if is_multiband else raster_data
            
            # For small datasets, use centroid method (faster)
            # For large datasets, consider using rasterio.mask (more accurate but slower)
            if len(geometries) < 10000:  # Threshold for method selection
                for geom in geometries:
                    try:
                        # Get centroid and sample
                        centroid = geom.centroid
                        col = int((centroid.x - transform.c) / transform.a)
                        row = int((centroid.y - transform.f) / transform.e)
                        
                        if 0 <= row < current_band.shape[0] and 0 <= col < current_band.shape[1]:
                            band_values.append(float(current_band[row, col]))
                        else:
                            band_values.append(np.nan)
                    except Exception:
                        band_values.append(np.nan)
            else:
                # For larger datasets, use mean value within polygon
                for geom in geometries:
                    try:
                        # Create temporary dataset for masking
                        temp_transform = transform
                        temp_shape = current_band.shape
                        
                        # Use rasterio mask to get values within polygon
                        masked_data, _ = rasterio_mask(
                            [{'data': current_band, 'transform': temp_transform}],
                            [geom],
                            crop=True,
                            nodata=np.nan
                        )
                        
                        if masked_data.size > 0:
                            band_values.append(float(np.nanmean(masked_data)))
                        else:
                            band_values.append(np.nan)
                    except Exception:
                        band_values.append(np.nan)
            
            gdf_data[f'band_{band_idx + 1}'] = band_values
        
        gdf = gpd.GeoDataFrame(gdf_data, crs=crs)
    else:
        # Single band case
        gdf = gpd.GeoDataFrame({
            'value': polygon_values,
            'geometry': geometries
        }, crs=crs)
    
    # Add source filename if provided
    if source_filename:
        gdf['source_file'] = source_filename
    
    # Save to file if requested
    if output_path:
        # Use optimized driver options
        if output_path.lower().endswith('.gpkg'):
            gdf.to_file(output_path, driver='GPKG')
        else:
            gdf.to_file(output_path)
        print(f"Vector file saved to: {output_path}")
    
    return gdf

def map_attributes_to_existing_features(base_gdf, new_gdf, source_name):
    """
    Map attributes from new GeoDataFrame to spatially matching features in base GeoDataFrame.
    
    Parameters:
    - base_gdf: Existing GeoDataFrame with base features
    - new_gdf: New GeoDataFrame with additional attributes to map
    - source_name: Name to use for the new attribute column
    
    Returns:
    - Updated base GeoDataFrame with new attributes
    """
    if new_gdf.empty:
        # Add empty column if no new data
        base_gdf[f'{source_name}_value'] = np.nan
        return base_gdf
    
    # Create spatial index for efficient lookups
    base_sindex = base_gdf.sindex
    
    # Initialize new attribute column
    new_values = np.full(len(base_gdf), np.nan)
    
    # For each new feature, find spatially matching base features
    for new_idx, new_row in new_gdf.iterrows():
        new_geom = new_row.geometry
        new_value = new_row.get('value', new_row.get('band_1', np.nan))
        
        # Find potential matches using spatial index
        possible_matches_idx = list(base_sindex.intersection(new_geom.bounds))
        
        if possible_matches_idx:
            # Check for exact spatial matches
            for base_idx in possible_matches_idx:
                base_geom = base_gdf.iloc[base_idx].geometry
                
                # Check if geometries are approximately equal using custom function
                if geometries_almost_equal(new_geom, base_geom):
                    new_values[base_idx] = new_value
                    break
                # Alternative: check for significant overlap
                elif new_geom.intersects(base_geom):
                    intersection_area = new_geom.intersection(base_geom).area
                    overlap_ratio = intersection_area / min(new_geom.area, base_geom.area)
                    if overlap_ratio > 0.9:  # 90% overlap threshold
                        new_values[base_idx] = new_value
                        break
    
    # Add new attribute column
    base_gdf[f'{source_name}_value'] = new_values
    
    # Count successful mappings
    mapped_count = np.sum(~np.isnan(new_values))
    total_new = len(new_gdf)
    
    print(f"  Mapped {mapped_count}/{total_new} features from {source_name}")
    
    return base_gdf

def stack_rasters(raster_paths, check_alignment=True):
    """Optimized raster stacking with optional alignment checking."""
    if not raster_paths:
        raise ValueError("No raster paths provided")
    
    # Read first raster to get reference properties
    with rasterio.open(raster_paths[0]) as ref_src:
        ref_meta = {
            'crs': ref_src.crs,
            'transform': ref_src.transform,
            'width': ref_src.width,
            'height': ref_src.height,
            'dtype': ref_src.dtypes[0]
        }
        
        # Pre-allocate array for better memory efficiency
        stack_array = np.empty((len(raster_paths), ref_src.height, ref_src.width), dtype=ref_meta['dtype'])
        stack_array[0] = ref_src.read(1)
    
    # Read remaining rasters
    for i, path in enumerate(raster_paths[1:], 1):
        with rasterio.open(path) as src:
            if check_alignment:
                if (src.crs != ref_meta['crs'] or 
                    src.transform != ref_meta['transform'] or 
                    src.width != ref_meta['width'] or 
                    src.height != ref_meta['height']):
                    raise ValueError(f"Raster {path} does not align spatially with reference raster")
            
            stack_array[i] = src.read(1)
    
    return stack_array

def get_cog_path(tif_path):
    """Generate COG path for a given TIFF file."""
    return os.path.splitext(tif_path)[0] + '_cog.tif'

def prefer_cog_files(tiff_files):
    """
    Efficiently prefer COG versions using set operations.
    """
    # Create sets for faster lookup
    all_files_set = set(tiff_files)
    preferred_files = []
    processed = set()
    
    for tif in tiff_files:
        if tif in processed:
            continue
            
        if tif.endswith('_cog.tif'):
            preferred_files.append(tif)
            processed.add(tif)
        else:
            cog_path = get_cog_path(tif)
            if cog_path in all_files_set:
                preferred_files.append(cog_path)
                processed.add(cog_path)
            else:
                preferred_files.append(tif)
            processed.add(tif)
    
    return preferred_files

def incremental_vector_processing(tiff_files, vector_output_path, mask_value=None, 
                                simplify_tolerance=None, combine_features=True):
    """
    Process rasters incrementally to avoid memory issues.
    
    For spatially aligned rasters, uses optimized value concatenation instead of spatial matching.
    Creates individual pixel polygons to ensure consistent spatial structure.
    
    Parameters:
    - tiff_files: List of TIFF file paths
    - vector_output_path: Output path for vector file
    - mask_value: Value to mask out
    - simplify_tolerance: Tolerance for polygon simplification
    - combine_features: If True, combine all features into one file; if False, create separate files
    
    Returns:
    - List of GeoDataFrames or single combined GeoDataFrame
    """
    gdfs = []
    combined_gdf = None
    reference_meta = None
    all_spatially_aligned = True
    
    # Check spatial alignment for all files first
    if combine_features and len(tiff_files) > 1:
        print("Checking spatial alignment of all rasters...")
        reference_meta = get_raster_metadata(tiff_files[0])
        
        for i, tif in enumerate(tiff_files[1:], 1):
            source_name = os.path.splitext(os.path.basename(tif))[0]
            is_aligned, current_meta = check_spatial_alignment(tif, reference_meta)
            if not is_aligned:
                all_spatially_aligned = False
                print(f"  Warning: {source_name} does not align spatially - using slower spatial matching")
                break
        
        if all_spatially_aligned:
            print("  All rasters are spatially aligned - using optimized processing")
            print("  Creating individual pixel polygons to prevent value merging")
    
    for i, tif in enumerate(tiff_files):
        print(f"Processing raster {i+1}/{len(tiff_files)}: {os.path.basename(tif)}")
        
        # Get source filename for attribution
        source_name = os.path.splitext(os.path.basename(tif))[0]
        
        if combine_features:
            if i == 0:
                # First raster becomes the base - use ordered conversion for consistency
                print(f"  Creating base vector layer from: {source_name}")
                combined_gdf = raster_to_vector_ordered(tif, mask_value=mask_value, 
                                                      simplify_tolerance=simplify_tolerance,
                                                      source_filename=source_name)
                
                if combined_gdf.empty:
                    print(f"  No polygons found in {source_name}")
                    continue
                
                # Rename value column to include source name
                if 'value' in combined_gdf.columns:
                    combined_gdf.rename(columns={'value': f'{source_name}_value'}, inplace=True)
                
                print(f"  Base layer created with {len(combined_gdf)} polygons")
                
            else:
                # Subsequent rasters
                if all_spatially_aligned:
                    # Fast path: directly extract values for aligned rasters
                    print(f"  Extracting values from aligned raster: {source_name}")
                    new_values = extract_raster_values_ordered(tif, combined_gdf, mask_value)
                    combined_gdf[f'{source_name}_value'] = new_values
                    
                    # Count non-NaN values
                    valid_count = np.sum(~np.isnan(new_values))
                    print(f"  Added {valid_count}/{len(new_values)} values from {source_name}")
                else:
                    # Slow path: spatial matching for non-aligned rasters
                    print(f"  Using spatial matching for: {source_name}")
                    gdf = raster_to_vector(tif, mask_value=mask_value, 
                                         simplify_tolerance=simplify_tolerance,
                                         source_filename=source_name)
                    
                    if gdf.empty:
                        print(f"  No polygons found in {source_name}")
                        combined_gdf[f'{source_name}_value'] = np.nan
                        continue
                    
                    # Map attributes from new raster to existing features
                    combined_gdf = map_attributes_to_existing_features(combined_gdf, gdf, source_name)
        else:
            # Keep separate - use standard conversion
            gdf = raster_to_vector(tif, mask_value=mask_value, 
                                 simplify_tolerance=simplify_tolerance,
                                 source_filename=source_name)
            
            if gdf.empty:
                print(f"  No polygons found in {source_name}")
                continue
            
            gdfs.append(gdf)
            
            # Save individual file
            if vector_output_path:
                base_name = os.path.splitext(vector_output_path)[0]
                ext = os.path.splitext(vector_output_path)[1] or '.gpkg'
                individual_output = f"{base_name}_{source_name}{ext}"
                gdf.to_file(individual_output)
                print(f"  Saved {len(gdf)} polygons to {individual_output}")
    
    if combine_features and combined_gdf is not None:
        # Clean up and save combined file
        # Remove helper columns before saving
        columns_to_drop = ['source_file']
        for col in columns_to_drop:
            if col in combined_gdf.columns:
                combined_gdf = combined_gdf.drop(columns=[col])
        
        if vector_output_path:
            if vector_output_path.lower().endswith('.gpkg'):
                combined_gdf.to_file(vector_output_path, driver='GPKG')
            else:
                combined_gdf.to_file(vector_output_path)
            print(f"Combined vector file saved to: {vector_output_path}")
            
            # Print summary of attributes
            value_columns = [col for col in combined_gdf.columns if col.endswith('_value')]
            print(f"Final dataset contains {len(combined_gdf)} features with {len(value_columns)} value attributes")
            
            if all_spatially_aligned:
                print("Processing completed using optimized aligned raster method with individual pixel polygons")
        
        return combined_gdf
    
    return gdfs if not combine_features else combined_gdf

def process_folder(folder, convert_cog=False, stack=False, to_vector=False, 
                   vector_output_path=None, mask_value=None, simplify_tolerance=None,
                   parallel_cog=False, check_alignment=True, combine_features=True,
                   incremental_processing=False):
    """
    Optimized folder processing with additional performance options.
    
    Parameters:
    - folder: Folder to search for TIFF files
    - convert_cog: Whether to convert TIFFs to COG format
    - stack: Whether to stack aligned rasters
    - to_vector: Whether to convert raster(s) to vector polygons
    - vector_output_path: Output path for vector file (if to_vector=True)
    - mask_value: Value to mask out when converting to vector
    - simplify_tolerance: Polygon simplification tolerance for faster processing
    - parallel_cog: Use parallel processing for COG conversion
    - check_alignment: Whether to check raster alignment (disable for speed if certain)
    - combine_features: If True, combine all vector features into one file/GDF; if False, keep separate
    - incremental_processing: If True, process rasters one by one to avoid memory issues
    
    Returns:
    - List of file paths, stacked array, GeoDataFrame, or list of GeoDataFrames depending on options
    """
    # Find all TIFF files
    all_tiffs = find_tiffs(folder)
    
    if not all_tiffs:
        print("No TIFF files found")
        return []
    
    # Always prefer COG versions if they exist
    tiffs = prefer_cog_files(all_tiffs)
    
    if convert_cog:
        files_to_convert = []
        converted_files = []
        
        for tif in tiffs:
            if tif.endswith('_cog.tif'):
                converted_files.append(tif)
                continue
                
            cog_path = get_cog_path(tif)
            
            if not os.path.exists(cog_path):
                files_to_convert.append((tif, cog_path))
            else:
                print(f"COG already exists for {tif}, skipping conversion.")
            
            converted_files.append(cog_path)
        
        # Convert files (optionally in parallel)
        if files_to_convert:
            if parallel_cog and len(files_to_convert) > 1:
                print(f"Converting {len(files_to_convert)} files to COG in parallel...")
                with ThreadPoolExecutor(max_workers=min(16, len(files_to_convert))) as executor:
                    executor.map(lambda x: convert_to_cog(x[0], x[1]), files_to_convert)
            else:
                for src, dst in files_to_convert:
                    print(f"Converting {src} to COG...")
                    convert_to_cog(src, dst)
        
        tiffs = converted_files
    
    # Handle vector processing
    if to_vector:
        if incremental_processing or (not stack and len(tiffs) > 1):
            # Use incremental processing for memory efficiency
            print("Using incremental processing to avoid memory issues...")
            return incremental_vector_processing(
                tiffs, vector_output_path, mask_value, 
                simplify_tolerance, combine_features
            )
        elif stack:
            # Stack first, then vectorize
            print("Stacking rasters first, then vectorizing...")
            stacked_data = stack_rasters(tiffs, check_alignment=check_alignment)
            return raster_to_vector(stacked_data, vector_output_path, mask_value, simplify_tolerance)
        else:
            # Single file
            return raster_to_vector(tiffs[0], vector_output_path, mask_value, simplify_tolerance)
    
    # Handle stacking without vectorization
    if stack:
        return stack_rasters(tiffs, check_alignment=check_alignment)
    
    return tiffs

# Alias for backwards compatibility
#process_folder = process_folder_optimized
#raster_to_vector = raster_to_vector_optimized
#stack_rasters = stack_rasters_optimized

# make find_tiffs() run in current folder
if __name__ == "__main__":
    folder = "."
    print("Finding TIFF files...")
    tiff_files = find_tiffs(folder)
    print(f"Found {len(tiff_files)} TIFF files.")

    print("Converting to COG...")
    process_folder(folder, convert_cog=True)

    print("Stacking aligned rasters...")
    try:
        stacked_data = process_folder(folder, stack=True)
        print(f"Stacked data shape: {stacked_data.shape}")
        
        print("Converting stacked raster to vector...")
        vector_gdf = process_folder(folder, stack=False, to_vector=True, 
                                  vector_output_path="stacked_polygons.shp")
        print(f"Created {len(vector_gdf)} polygons")
        
    except ValueError as e:
        print(e)

In [6]:
# folder = r"G:\arcgis_prosjekt_publisert_azure\servicedata\naturskog_v1\project_9ae905e7-a7a5-43ed-b468-63b29232d4b8\delivery_6baba287-82a7-4c32-b481-fa1a864b046a"
folder = r"C:\Users\endofs\Downloads\project_9ae905e7-a7a5-43ed-b468-63b29232d4b8\delivery_6baba287-82a7-4c32-b481-fa1a864b046a"

In [7]:
# Example usage with combined features
result = process_folder(
    folder=".",
    to_vector=True,
    combine_features=True,  # Maps attributes instead of concatenating
    incremental_processing=True,
    vector_output_path="combined_analysis.gpkg"
)

# Result will have:
# - Same number of features as first raster
# - Columns: geometry, raster1_value, raster2_value, raster3_value, etc.
# - Spatial alignment enforced

Using incremental processing to avoid memory issues...
Checking spatial alignment of all rasters...
  All rasters are spatially aligned - using optimized processing
  Creating individual pixel polygons to prevent value merging
Processing raster 1/3: random_geotiff_1_cog.tif
  Creating base vector layer from: random_geotiff_1_cog
  Created 10000 individual pixel polygons (no merging)
  Base layer created with 10000 polygons
Processing raster 2/3: random_geotiff_2_cog.tif
  Extracting values from aligned raster: random_geotiff_2_cog
  Added 10000/10000 values from random_geotiff_2_cog
Processing raster 3/3: random_geotiff_3_cog.tif
  Extracting values from aligned raster: random_geotiff_3_cog
  Added 10000/10000 values from random_geotiff_3_cog
Combined vector file saved to: combined_analysis.gpkg
Final dataset contains 10000 features with 3 value attributes
Processing completed using optimized aligned raster method with individual pixel polygons
  Created 10000 individual pixel polygons

In [ ]:
# Example usage with combined features
folder = r"C:\Users\endofs\Downloads\project_9ae905e7-a7a5-43ed-b468-63b29232d4b8\delivery_6baba287-82a7-4c32-b481-fa1a864b046a"

result = process_folder(
    folder=folder,
    convert_cog=True, # Convert to COG if not already, especcially beneficial for large rasters
    parallel_cog=True, # Use parallel processing for COG conversion
    to_vector=True, # Convert to vector polygons
    combine_features=True,  # Maps attributes instead of concatenating
    incremental_processing=True, # to avoid memory issues
    vector_output_path="naturskog_v1.gpkg"
)

Using incremental processing to avoid memory issues...
Checking spatial alignment of all rasters...
  All rasters are spatially aligned - using optimized processing
  Creating individual pixel polygons to prevent value merging
Processing raster 1/6: naturskog_binaer_cog.tif
  Creating base vector layer from: naturskog_binaer_cog
  All rasters are spatially aligned - using optimized processing
  Creating individual pixel polygons to prevent value merging
Processing raster 1/6: naturskog_binaer_cog.tif
  Creating base vector layer from: naturskog_binaer_cog


In [ ]:
# Clear all variables from memory
%reset -f